In [1]:
# coding:u8
import json
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
import gc
from gensim.models import Word2Vec
from sklearn.model_selection import KFold
from sklearn.externals import joblib

import pickle

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
train_data = []
train_data_dict = {}
with open('data/raw_data/train.json', 'r') as f:
    for line in f:
        raw = eval(str(json.loads(line)).lower())
        train_data.append(raw)
        #train_data_dict[raw['text_id']] = raw
kb = {}
with open('data/raw_data/kb_data', 'r') as f:
    for line in f:
        item = eval(str(json.loads(line)).lower())
        kb[item['subject_id']] = item

name_id = {}
for kb_id in kb:
    for item in kb[kb_id]['alias']:
        if item not in name_id:
            name_id[item] = [kb_id]
        else:
            name_id[item].append(kb_id)
    if kb[kb_id]['subject'] not in name_id:
        name_id[kb[kb_id]['subject']] = [kb_id]
    else:
        name_id[kb[kb_id]['subject']].append(kb_id)
for id in name_id:
    name_id[id] = sorted(list(set(name_id[id])))

In [3]:
testfile_name = 'data/raw_data/develop.json'
raw_submit = []
text_idto_text = {}
with open(testfile_name, 'r') as f:
    for line in tqdm(f):
        s = json.loads(line)
        text_idto_text[s['text_id']] = s['text']
        raw_submit.append(s)


10000it [00:00, 14443.65it/s]


In [4]:
data = pd.read_pickle('data/step1_5_test.pkl')
data2 = pd.read_pickle('data/step2_test.pkl')
data3 = pd.read_pickle('data/step5_test.pkl')

In [5]:
for c in data2.columns:
    if c not in data.columns:
        data[c] = data2[c]
for c in data3.columns:
    if c not in data.columns:
        data[c] = data3[c]

In [6]:
predictor=data.columns
exclude = ['text_id',
 'kb_id',
 'train_mention',
 'label',
 'm_id','type']
# exclude = []
temp = []
for x in predictor:
    if x not in exclude :temp.append(x)
predictor2 = temp 

In [7]:
gbm_pickle = joblib.load('model_link/lgb.pkl')
assert len(gbm_pickle.feature_name()) == len(predictor2)

In [8]:
data['link_grade'] = gbm_pickle.predict(data[predictor2])

In [25]:
# 过滤低预测值数据
valid_data = data[data.link_grade>0.35]
submit = valid_data.groupby('m_id',as_index=False)['link_grade'].agg({'label_id_max':'idxmax'})
m = submit.shape[0]

In [26]:
submit.shape

(26894, 2)

In [20]:
data[['text_id','kb_id','train_mention','label_mean','label_count','m_label_mean',
                                 'm_label_count','link_grade']].head(20)

,text_id,kb_id,train_mention,label_mean,label_count,m_label_mean,m_label_count,link_grade
0,1,113234,忏悔录,0.055556,18.0,0.055556,18.0,0.089671
1,1,119304,忏悔录,0.833333,18.0,0.833333,18.0,0.716872
2,1,372331,忏悔录,0.111111,18.0,0.111111,18.0,0.035812
3,1,38456,忏悔录,0.000000,18.0,0.000000,18.0,0.010338
4,1,386314,忏悔录,0.000000,18.0,0.000000,18.0,0.003495
5,2,303980,最终幻想世界,1.000000,5.0,1.000000,5.0,0.930052
6,2,52738,最终幻想世界,0.000000,5.0,0.000000,5.0,0.024500
7,3,64062,谢文东2,NaN,NaN,NaN,NaN,0.915135
8,3,69067,谢文东2,NaN,NaN,NaN,NaN,0.091836
9,3,202048,电视剧,0.999320,2940.0,0.999320,2940.0,0.999350


In [22]:
valid_data.shape

(26894, 609)

In [11]:
hit = valid_data.loc[submit['label_id_max'],:]
print(hit.head())
valid_data = hit

    create_works  end_with_line  end_with_line2  end_with_point  \
1              1              0               0               0   
5              1              0               0               0   
7              1              1               1               0   
9              0              1               1               0   
12             0              0               0               0   

    end_with_shuminghao  entity_common   kb_id  label  len_mention    m_id  \
1                     1              1  119304     -1            3  257734   
5                     1              0  303980     -1            6  257735   
7                     1              1   64062     -1            4  257736   
9                     1              1  202048     -1            3  257737   
12                    0              1  382390     -1            2  257738   

    ...  m_rank_mse_distance  mid_cos_distance_var  mid_cos_distance_mean  \
1   ...                  6.0              0.881605 

In [12]:
kbid_finder = {}
for index,row in valid_data.iterrows():
    kbid_finder[str(row['text_id'])+row['train_mention']] = str(row['kb_id'])

In [13]:
# 戴安楠ner
ner_result = pd.read_csv('result/el_dev_result.csv', sep='\t')
ner_result['offset'] = ner_result['offset'].apply(lambda x: eval(x.lower()))
ner_result['mention'] = ner_result['mention'].apply(lambda x: eval(x.lower()))
ner_result = ner_result.rename(columns={'mention': 'pred', 'offset': 'pos'})
test_data = []
for index, row in ner_result.iterrows():
    item = {}
    item['text_id'] = row['text_id']
    item['text'] = row['text']
    item['mention_data'] = []
    for i, p in enumerate(row['pos']):
        if row['pred'][i] not in name_id:continue
        item['mention_data'].append({'mention': row['pred'][i], 'offset': str(p)})
    test_data.append(item)

In [21]:
kbid_finder

{'1忏悔录': '119304',
 '2最终幻想世界': '303980',
 '3谢文东2': '64062',
 '3电视剧': '202048',
 '4适合': '382390',
 '43岁': '207291',
 '5东日本大地震': '20128',
 '6机会来了': '219277',
 '6高清视频': '338774',
 '7结局': '335878',
 '8奇幻加冕礼': '317158',
 '9韩剧': '237537',
 '9戏子': '394766',
 '9影评': '22265',
 '10紫萱': '141031',
 '10林业平': '32716',
 '10原型': '290312',
 '11双节棍': '74729',
 '11教程': '147300',
 '11教育': '160301',
 '11高清视频': '338774',
 '12发际红': '99972',
 '12百度影音': '218566',
 '12优酷': '202883',
 '12土豆网': '18047',
 '132015': '63363',
 '14互联网': '312651',
 '15分手了就不要再想起我': '181308',
 '17谭松韵': '27406',
 '17熊梓淇': '137009',
 '17浪花一朵朵': '338573',
 '17花絮': '124757',
 '18封面': '48116',
 '18人物': '152760',
 '18王宏伟': '221497',
 '19中国': '241286',
 '19枫泾': '67621',
 '19副部长': '22237',
 '20绯闻女孩': '408111',
 '20大结局': '300787',
 '20剧透': '337606',
 '21音乐': '130934',
 '22政策': '365118',
 '22活动': '402599',
 '23评分': '171465',
 '23国产片': '288297',
 '23国产剧': '300420',
 '24蒙面歌王': '336100',
 '24总决赛': '134060',
 '24野草': '141674',
 '24谭维维': '251054',
 '2

In [14]:
for item in test_data:
    # if item['kb_id']=='nil':continue
    item['text_id']=str(item['text_id'])
    item['text']=text_idto_text[item['text_id']]
    for m in item['mention_data']:
        try:
            m['kb_id']=kbid_finder[str(item['text_id'])+m['mention']]
        except:
            print(item,m)
            m['kb_id'] = 'NIL' # 去掉某些ner
        m['mention'] = item['text'][int(m['offset']):int(m['offset'])+len(m['mention'])]
submit_data = test_data

{'text_id': '4', 'text': '进口喜宝奶粉有适合3岁以上的吗', 'mention_data': [{'mention': '进口', 'offset': '0'}, {'mention': '喜宝奶粉', 'offset': '2'}, {'mention': '适合', 'offset': '7'}]} {'mention': '进口', 'offset': '0'}
{'text_id': '4', 'text': '进口喜宝奶粉有适合3岁以上的吗', 'mention_data': [{'mention': '进口', 'offset': '0', 'kb_id': 'NIL'}, {'mention': '喜宝奶粉', 'offset': '2'}, {'mention': '适合', 'offset': '7'}]} {'mention': '喜宝奶粉', 'offset': '2'}
{'text_id': '6', 'text': '《机会来了》 - 完整版-原创-高清视频', 'mention_data': [{'mention': '机会来了', 'offset': '1', 'kb_id': '219277'}, {'mention': '视频', 'offset': '18'}]} {'mention': '视频', 'offset': '18'}
{'text_id': '8', 'text': '【评】关于《奇幻加冕礼》自己的看法_snh48吧', 'mention_data': [{'mention': '奇幻加冕礼', 'offset': '6', 'kb_id': '317158'}, {'mention': '看法', 'offset': '15'}]} {'mention': '看法', 'offset': '15'}
{'text_id': '13', 'text': '《上海纪实-档案》 20151029 76号魔窟 第五集 桃色...', 'mention_data': [{'mention': '76号魔窟', 'offset': '19'}]} {'mention': '76号魔窟', 'offset': '19'}
{'text_id': '16', 'text': '一首《黑熊精》送给你们-原

{'text_id': '5732', 'text': '大班绘本《克里克塔》', 'mention_data': [{'mention': '大班', 'offset': '0'}, {'mention': '绘本', 'offset': '2'}, {'mention': '克里克塔', 'offset': '5'}]} {'mention': '大班', 'offset': '0'}
{'text_id': '5732', 'text': '大班绘本《克里克塔》', 'mention_data': [{'mention': '大班', 'offset': '0', 'kb_id': 'NIL'}, {'mention': '绘本', 'offset': '2'}, {'mention': '克里克塔', 'offset': '5'}]} {'mention': '绘本', 'offset': '2'}
{'text_id': '5741', 'text': '如何评价小说《藏婚》,小说完结了吗?', 'mention_data': [{'mention': '小说', 'offset': '4', 'kb_id': '64589'}, {'mention': '藏婚', 'offset': '7'}, {'mention': '小说', 'offset': '11'}]} {'mention': '藏婚', 'offset': '7'}
{'text_id': '5749', 'text': '《婚巢》收视飘红 王柠为戏不惜毁形象', 'mention_data': [{'mention': '婚巢', 'offset': '1', 'kb_id': '116612'}, {'mention': '收视', 'offset': '4'}, {'mention': '王柠', 'offset': '9'}, {'mention': '形象', 'offset': '16'}]} {'mention': '收视', 'offset': '4'}
{'text_id': '5750', 'text': '宝丰马街书会擂台 山东快书《十字坡》-搞笑-', 'mention_data': [{'mention': '擂台', 'offset': '6', 'kb_id'

In [15]:
new_submit = []
for item in submit_data:
    new_item = {}
    new_item['text_id'] = item['text_id']
    new_item['text'] = item['text']
    new_item['mention_data'] = []
    for m in item['mention_data']:
        if m['kb_id']=='NIL':continue
        new_item['mention_data'].append(m)
    new_submit.append(new_item)

In [16]:
import json
with open('submit/result_7.json', 'w') as f:
    for line in new_submit:
        json.dump(line,f,ensure_ascii=False)
        f.write('\n')

In [17]:
data[['text_id','kb_id','train_mention','label_mean','label_count','m_label_mean',
                                 'm_label_count','link_grade']]

,text_id,kb_id,train_mention,label_mean,label_count,m_label_mean,m_label_count,link_grade
0,1,113234,忏悔录,0.055556,18.0,0.055556,18.0,0.089671
1,1,119304,忏悔录,0.833333,18.0,0.833333,18.0,0.716872
2,1,372331,忏悔录,0.111111,18.0,0.111111,18.0,0.035812
3,1,38456,忏悔录,0.000000,18.0,0.000000,18.0,0.010338
4,1,386314,忏悔录,0.000000,18.0,0.000000,18.0,0.003495
5,2,303980,最终幻想世界,1.000000,5.0,1.000000,5.0,0.930052
6,2,52738,最终幻想世界,0.000000,5.0,0.000000,5.0,0.024500
7,3,64062,谢文东2,NaN,NaN,NaN,NaN,0.915135
8,3,69067,谢文东2,NaN,NaN,NaN,NaN,0.091836
9,3,202048,电视剧,0.999320,2940.0,0.999320,2940.0,0.999350
